# Reading Data from external sources
<img src="https://user-images.githubusercontent.com/7065401/68501079-0695df00-023c-11ea-841f-455dac84a089.jpg"
    style="width:400px; float: right; margin: 0 40px 40px 40px;"></img>
We have seen that pandas is powerful enough to read data from multiple data source like csv, db, etc. Here we will see in more depth how we can use pandas to read and write data from multiple sources.

Rather than creating `Series` or `DataFrames` strutures from scratch, or even from Python core sequences or `ndarrays`, the most typical use of **pandas** is based on the loading of information from files or sources of information for further exploration, transformation and analysis.

In [ ]:
import pandas as pd

## Reading CSV and TXT files

In this lecture we'll learn how to read comma-separated values files (.csv) and raw text files (.txt) into pandas `DataFrame`s.

### Reading data with Python

As we saw on previous courses we can read data simply using Python.

When you want to work with a file, the first thing to do is to open it. This is done by invoking the `open()` built-in function.

`open()` has a single required argument that is the path to the file and has a single return, the file object.

The `with` statement automatically takes care of closing the file once it leaves the `with` block, even in cases of error.

In [ ]:
with open('data/btc-market-price-csv-read.csv', 'r') as fp:
    print(fp)

Once the file is opened, we can read its content as follows:

In [ ]:
with open('data/btc-market-price-csv-read.csv', 'r') as fp:
    for index, line in enumerate(fp.readlines()):
        # read just the first 10 lines
        if (index < 10):
            print(index, line)

How can we process the data read from the file using pure Python? It involves a lot of manual work, for example, splitting the values by the correct separator:

In [ ]:
with open('data/btc-market-price-csv-read.csv', 'r') as fp:
    for index, line in enumerate(fp.readlines()):
        # read just the first 10 lines
        if (index < 10):
            timestamp, price = line.split(',')
            print(f"{timestamp}: ${price}")

But what happens if the separator is unknown, like in the file `exam_review.csv`:

In [ ]:
!head data/exam_review.csv

In this case, the separator is not a _comma_, but the `>` sign. It's still a "CSV", although not technically separated by commas.

#### The `csv` module

Python includes the builtin module `csv` that helps a little bit more with the process of reading CSVs:

In [ ]:
import csv

In [ ]:
with open('data/btc-market-price-csv-read.csv', 'r') as fp:
    reader = csv.reader(fp)
    for index, (timestamp, price) in enumerate(reader):
        # read just the first 10 lines
        if (index < 10):
            print(f"{timestamp}: ${price}")

The `csv` modules takes care of splitting the file using a given separator (called `delimiter`) and creating an iterator for us.

In [ ]:
with open('data/exam_review.csv', 'r') as fp:
    reader = csv.reader(fp, delimiter='>')  # special delimiter
    next(reader)  # skipping header
    for index, values in enumerate(reader):
        if not values:
            continue  # skip empty lines
        fname, lname, age, math, french = values
        print(f"{fname} {lname} (age {age}) got {math} in Math and {french} in French")

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Reading data with Pandas

Probably one of the most recurrent types of work for data analysis: public data sources, logs, historical information tables, exports from databases. So the pandas library offers us functions to read and write files in multiple formats like CSV, JSON, XML and Excel's XLSX, all of them creating a `DataFrame` with the information read from the file.

We'll learn how to read different type of data including:
- CSV files (.csv)
- Raw text files (.txt)
- JSON data from a file and from an API
- Data from a SQL query over a database

There are many other available reading functions as the following table shows:

![pandas read data table](https://user-images.githubusercontent.com/7065401/68400151-51d5c200-0156-11ea-9732-aa00400c8e4b.png)

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### The `read_csv` method

The first method we'll learn is **read_csv**, that let us read comma-separated values (CSV) files and raw text (TXT) files into a `DataFrame`.

The `read_csv` function is extremely powerful and you can specify a very broad set of parameters at import time that allow us to accurately configure how the data will be read and parsed by specifying the correct structure, enconding and other details. The most common parameters are as follows:

- `filepath`: Path of the file to be read.
- `sep`: Character(s) that are used as a field separator in the file.
- `header`: Index of the row containing the names of the columns (None if none).
- `index_col`: Index of the column or sequence of indexes that should be used as index of rows of the data.
- `names`: Sequence containing the names of the columns (used together with header = None).
- `skiprows`: Number of rows or sequence of row indexes to ignore in the load.
- `na_values`: Sequence of values that, if found in the file, should be treated as NaN.
- `dtype`: Dictionary in which the keys will be column names and the values will be types of NumPy to which their content must be converted.
- `parse_dates`: Flag that indicates if Python should try to parse data with a format similar to dates as dates. You can enter a list of column names that must be joined for the parsing as a date.
- `date_parser`: Function to use to try to parse dates.
- `nrows`: Number of rows to read from the beginning of the file.
- `skip_footer`: Number of rows to ignore at the end of the file.
- `encoding`: Encoding to be expected from the file read.
- `squeeze`: Flag that indicates that if the data read only contains one column the result is a Series instead of a DataFrame.
- `thousands`: Character to use to detect the thousands separator.
- `decimal`: Character to use to detect the decimal separator.
- `skip_blank_lines`: Flag that indicates whether blank lines should be ignored.

> Full `read_csv` documentation can be found here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html.

In this case we'll try to read our `btc-market-price.csv` CSV file using different parameters to parse it correctly.

This file contains records of the mean price of Bitcoin per date.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Reading our first CSV file

Everytime we call `read_csv` method, we'll need to pass an explicit `filepath` parameter indicating the path where our CSV file is.

Any valid string path is acceptable. The string could be a URL. Valid URL schemes include HTTP, FTP, S3, and file. For file URLs, a host is expected. A local file could be: `file://localhost/path/to/table.csv`.

For example we can use `read_csv` method to load data directly from an URL:

In [ ]:
csv_url = "https://raw.githubusercontent.com/datasets/gdp/master/data/gdp.csv"

pd.read_csv(csv_url).head()

Or just use a local file:

In [ ]:
df = pd.read_csv('data/btc-market-price-csv-read.csv')

df.head()

In this case we let pandas infer everything related to our data, but in most of the cases we'll need to explicitly tell pandas how we want our data to be loaded. To do that we use parameters.

Let's see how theses parameters work.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### First row behaviour with `header` parameter

The CSV file we're reading has only two columns: `Timestamp` and `Price`. It doesn't have a header. Pandas automatically assigned the first row of data as headers, which is incorrect. We can overwrite this behavior with the `header` parameter.

In [ ]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None)

In [ ]:
df.head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Missing values with `na_values` parameter

We can define a `na_values` parameter with the values we want to be recognized as NA/NaN. In this case empty strings `''`, `?` and `-` will be recognized as null values.

In [ ]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None,
                 na_values=['', '?', '-'])

In [ ]:
df.head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Column names using `names` parameter

We'll add that columns names using the `names` parameter. 

In [ ]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None,
                 na_values=['', '?', '-'],
                 names=['Timestamp', 'Price'])

In [ ]:
df.head()

In [ ]:
df.info()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Column types using `dtype` parameter


Without using the `dtype` parameter pandas will try to figure it out the type of each column automatically. We can use `dtype` parameter to force pandas to use certain dtype.

In this case we'll force the `Price` column to be `float`.

In [ ]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None,
                 na_values=['', '?', '-'],
                 names=['Timestamp', 'Price'],
                 dtype={'Price': 'float'})

In [ ]:
df.head()

In [ ]:
df.dtypes

The `Timestamp` column was interpreted as a regular string (`object` in pandas notation), we can parse it manually using a vectorized operation as we saw on previous courses.

We'll parse `Timestamp` column to `Datetime` objects using `to_datetime` method:

In [ ]:
pd.to_datetime(df['Timestamp']).head()

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [ ]:
df.head()

In [ ]:
df.dtypes

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Date parser using `parse_dates` parameter

Another way of dealing with `Datetime` objects is using `parse_dates` parameter with the position of the columns with dates.

In [ ]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None,
                 na_values=['', '?', '-'],
                 names=['Timestamp', 'Price'],
                 dtype={'Price': 'float'},
                 parse_dates=[0])

In [ ]:
df.head()

In [ ]:
df.dtypes

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Adding index to our data using `index_col` parameter

By default, pandas will automatically assign a numeric autoincremental index or row label starting with zero. You may want to leave the default index as such if your data doesn’t have a column with unique values that can serve as a better index. In case there is a column that you feel would serve as a better index, you can override the default behavior by setting `index_col` property to a column. It takes a numeric value representing the index or a string of the column name for setting a single column as index or a list of numeric values or strings for creating a multi-index.

In our data, we are choosing the first column, `Timestamp`, as index (index=0) by passing zero to the `index_col` argument.

In [ ]:
df = pd.read_csv('data/btc-market-price-csv-read.csv',
                 header=None,
                 na_values=['', '?', '-'],
                 names=['Timestamp', 'Price'],
                 dtype={'Price': 'float'},
                 parse_dates=[0],
                 index_col=[0])

In [ ]:
df.head()

In [ ]:
df.dtypes

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### A more challenging parsing

Now we'll read another CSV file. This file has the following columns:

- `first_name`
- `last_name`
- `age`
- `math_score`
- `french_score`
- `next_test_date`

Let's read it and see how it looks like.

In [ ]:
exam_df = pd.read_csv('data/exam_review.csv')

In [ ]:
exam_df

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Custom data delimiters using `sep` parameter

We can define which delimiter to use by using the `sep` parameter. If we don't use the `sep` parameter, pandas will automatically detect the separator.

In most of the CSV files separator will be comma (`,`) and will be automatically detected. But we can find files with other separators like semicolon (`;`), tabs (`\t`, specially on TSV files), whitespaces or any other special character.

In this case the separator is a `>` character.

In [ ]:
exam_df = pd.read_csv('data/exam_review.csv',
                      sep='>')

In [ ]:
exam_df

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Custom data encoding

Files are stored using different "encodings". You've probably heard about ASCII, UTF-8, latin1, etc.

While reading data custom encoding can be defined with the `encoding` parameter.

- `encoding='UTF-8'`: will be used if data is UTF-8 encoded.
- `encoding='iso-8859-1'`: will be used if data is ISO/IEC 8859-1 ("extended ASCII") encoded.

In our case we don't need a custom enconding as data is properly loaded.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Custom numeric `decimal` and `thousands` character  

The decimal and thousands characters could change between datasets. If we have a column containing a comma (`,`) to indicate the decimal or thousands place, then this column would be considered a string and not numeric.

In [ ]:
exam_df = pd.read_csv('data/exam_review.csv',
                      sep='>')

In [ ]:
exam_df

In [ ]:
exam_df[['math_score', 'french_score']].dtypes

To solve that, ensuring such columns are interpreted as integer values, we'll need to use the `decimal` and/or `thousands` parameters to indicate correct decimal and/or thousands indicators.

In [ ]:
exam_df = pd.read_csv('data/exam_review.csv',
                      sep='>',
                      decimal=',')

In [ ]:
exam_df

In [ ]:
exam_df[['math_score', 'french_score']].dtypes

Let's see what happens with the `thousands` parameter:

In [ ]:
pd.read_csv('data/exam_review.csv',
            sep='>',
            thousands=',')

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Excluding specific rows

We can use the `skiprows` to:

- Exclude reading specified number of rows from the beginning of a file, by passing an integer argument. **This removes the header too**.
- Skip reading specific row indices from a file, by passing a list containing row indices to skip.

In [ ]:
exam_df = pd.read_csv('data/exam_review.csv',
                      sep='>',
                      decimal=',')

In [ ]:
exam_df

To skip reading the first 2 rows from this file, we can use `skiprows=2`:

In [ ]:
pd.read_csv('data/exam_review.csv',
            sep='>',
            skiprows=2)

As the header is considered as the first row, to skip reading data rows 1 and 3, we can use `skiprows=[1,3]`:

In [ ]:
exam_df = pd.read_csv('data/exam_review.csv',
                      sep='>',
                      decimal=',',
                      skiprows=[1,3])

In [ ]:
exam_df

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Get rid of blank lines

The `skip_blank_lines` parameter is set to `True` so blank lines are skipped while we read files.

If we set this parameter to `False`, then every blank line will be loaded with `NaN` values into the `DataFrame`.

In [ ]:
pd.read_csv('data/exam_review.csv',
            sep='>',
            skip_blank_lines=False)

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Loading specific columns

We can use the `usecols` parameter when we want to load just specific columns and not all of them.

Performance wise, it is better because instead of loading an entire dataframe into memory and then deleting the not required columns, we can select the columns that we’ll need, while loading the dataset itself.

As a parameter to `usecols`, you can pass either a list of strings corresponding to the column names or a list of integers corresponding to column index.

In [ ]:
pd.read_csv('data/exam_review.csv',
            usecols=['first_name', 'last_name', 'age'],
            sep='>')

Or using just the column position:

In [ ]:
pd.read_csv('data/exam_review.csv',
            usecols=[0, 1, 2],
            sep='>')

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Using a `Series` instead of `DataFrame`

If the parsed data only contains one column then we can return a Series by setting the `squeeze` parameter to `True`.

In [ ]:
exam_test_1 = pd.read_csv('data/exam_review.csv',
                          sep='>',
                          usecols=['last_name'])

In [ ]:
exam_test_1

In [ ]:
type(exam_test_1)

In [ ]:
exam_test_2 = pd.read_csv('data/exam_review.csv',
                          sep='>',
                          usecols=['last_name'],
                          squeeze=True)

In [ ]:
exam_test_2

In [ ]:
type(exam_test_2)

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Save to CSV file

Finally we can also save our `DataFrame` as a CSV file.

In [ ]:
exam_df

We can simply generate a CSV string from our `DataFrame`:

In [ ]:
exam_df.to_csv()

Or specify a file path where we want our generated CSV code to be saved:

In [ ]:
exam_df.to_csv('data/out-csv.csv')

In [ ]:
pd.read_csv('data/out-csv.csv')

In [ ]:
exam_df.to_csv('data/out-csv.csv',
               index=None)

In [ ]:
pd.read_csv('data/out-csv.csv')

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Reading data from relational databases

In this lesson you will learn how to read SQL queries and relational database tables into `DataFrame` objects using pandas. Also, we'll take a look at different techniques to persist that pandas `DataFrame` objects to database tables.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Read data from SQL database

Reading data from SQL relational databases is fairly simple and pandas support a variety of methods to deal with it.

We'll start with an example using SQLite, as it's a builtin Python package, and we don't need anything extra installed.

In [ ]:
import sqlite3

In order to work with a SQLite database from Python, we first have to connect to it. We can do that using the connect function, which returns a `Connection` object.

We'll use the following database structure:

<center>
<img src="./Images/chinook-dbstructure.png" width="700px" align="center" />
</center>

In [ ]:
conn = sqlite3.connect('data/chinook.db')

Once we have a `Connection` object, we can then create a `Cursor` object. Cursors allow us to execute SQL queries against a database:

In [ ]:
cur = conn.cursor()

The `Cursor` created has a method `execute`, which will receive SQL parameters to run against the database.

The code below will fetch the first `5` rows from the `employees` table:

In [ ]:
cur.execute('SELECT * FROM employees LIMIT 5;')

You may have noticed that we didn't assign the result of the above query to a variable. This is because we need to run another command to actually fetch the results.

We can use the `fetchall` method to fetch all of the results of a query:

In [ ]:
results = cur.fetchall()

In [ ]:
results

As you can see, the results are returned as a list of tuples. Each tuple corresponds to a row in the database that we accessed. Dealing with data this way is painful.

We'd need to manually add column headers, and manually parse the data. Luckily, the pandas library has an easier way, which we'll look at in the next section.

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df.head()

Before we move on, it's good practice to close `Connection` objects and `Cursor` objects that are open. This prevents the SQLite database from being locked. When a SQLite database is locked, you may be unable to update the database, and may get errors. We can close the Cursor and the Connection like this:

In [ ]:
cur.close()
conn.close()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Using pandas `read_sql` method

We can use the pandas `read_sql` function to read the results of a SQL query directly into a pandas `DataFrame`. The code below will execute the same query that we just did, but it will return a `DataFrame`. It has several advantages over the query we did above:

- It doesn't require us to create a `Cursor` object or call `fetchall` at the end.
- It automatically reads in the names of the headers from the table.
- It creates a `DataFrame`, so we can quickly explore the data.

In [ ]:
conn = sqlite3.connect('data/chinook.db')

In [ ]:
df = pd.read_sql('SELECT * FROM employees;', conn)

In [ ]:
df.head()

In [ ]:
df = pd.read_sql('SELECT * FROM employees;', conn,
                 index_col='EmployeeId',
                 parse_dates=['BirthDate', 'HireDate'])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['ReportsTo'].isna().sum()

In [ ]:
df['ReportsTo'].mean()

In [ ]:
df['ReportsTo'] > 1.75

In [ ]:
df['City'] = df['City'].astype('category')

In [ ]:
df.info()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Using pandas `read_sql_query` method

It turns out that the `read_sql` method we saw above is just a wrapper around `read_sql_query` and `read_sql_table`.

We can get the same result using `read_sql_query` method:

In [ ]:
conn = sqlite3.connect('data/chinook.db')

In [ ]:
df = pd.read_sql_query('SELECT * FROM employees LIMIT 5;', conn)

In [ ]:
df.head()

In [ ]:
df = pd.read_sql_query('SELECT * FROM employees;', conn,
                       index_col='EmployeeId',
                       parse_dates=['BirthDate', 'HireDate'])

In [ ]:
df.head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Using `read_sql_table` method

`read_sql_table` is a useful function, but it works only with [SQLAlchemy](https://www.sqlalchemy.org/), a Python SQL Toolkit and Object Relational Mapper.

This is just a demonstration of its usage where we read the whole `employees` table.

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///data/chinook.db')

connection = engine.connect()

In [ ]:
df = pd.read_sql_table('employees', con=connection)

In [ ]:
df.head()

In [ ]:
df = pd.read_sql_table('employees', con=connection,
                       index_col='EmployeeId',
                       parse_dates=['BirthDate', 'HireDate'])

In [ ]:
df.head()

In [ ]:
connection.close()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Create tables from `DataFrame` objects

Finally we can persist `DataFrame` objects we've working on in a database using the pandas `to_sql` method.

Although it is easy to implement, it could be a very slow process.

In [ ]:
df.head()

_Drop the table if needed_

In [ ]:
df.to_sql?

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute('DROP TABLE IF EXISTS employees2;')

In [ ]:
cur.close()

In [ ]:
df.to_sql('employees2', conn)

In [ ]:
pd.read_sql_query('SELECT * FROM employees2;', conn).head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Custom behavior

The `if_exists` parameter define how to behave if the table already exists and adds a ton of flexibility, letting you decide wheather to `replace` current table data, `append` new data at the end, or simply `fail` if table already exists.

In [ ]:
pd.DataFrame().to_sql('employees2',
                      conn,
                      if_exists='replace')

In [ ]:
pd.read_sql_query('SELECT * FROM employees2;', conn).head()

In [ ]:
df.to_sql('employees2',
          conn,
          if_exists='replace')

In [ ]:
pd.read_sql_query('SELECT * FROM employees2;', conn).head()

In [ ]:
conn.close()

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Reading Excel files

In this lecture we'll learn how to read Excel files (.xlsx) and its sheets into a pandas `DataFrame`s, and how to export that `DataFrame`s to different sheets and Excel files using the pandas `ExcelWriter` and `to_excel` methods.

In [ ]:
!head data/products.xlsx

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### The `read_excel` method

We'll begin with the **read_excel** method, that let us read Excel files into a `DataFrame`.

This method supports both XLS and XLSX file extensions from a local filesystem or URL and has a broad set of parameters to configure how the data will be read and parsed. These parameters are very similar to the parameters we saw on previous lectures where we introduced the `read_csv` method. The most common parameters are as follows:

- `filepath`: Path of the file to be read.
- `sheet_name`: Strings are used for sheet names. Integers are used in zero-indexed sheet positions. Lists of strings/integers are used to request multiple sheets. Specify None to get all sheets.
- `header`: Index of the row containing the names of the columns (None if none).
- `index_col`: Index of the column or sequence of indexes that should be used as index of rows of the data.
- `names`: Sequence containing the names of the columns (used together with header = None).
- `skiprows`: Number of rows or sequence of row indexes to ignore in the load.
- `na_values`: Sequence of values that, if found in the file, should be treated as NaN.
- `dtype`: Dictionary in which the keys will be column names and the values will be types of NumPy to which their content must be converted.
- `parse_dates`: Flag that indicates if Python should try to parse data with a format similar to dates as dates. You can enter a list of column names that must be joined for the parsing as a date.
- `date_parser`: Function to use to try to parse dates.
- `nrows`: Number of rows to read from the beginning of the file.
- `skip_footer`: Number of rows to ignore at the end of the file.
- `squeeze`: Flag that indicates that if the data read only contains one column the result is a Series instead of a DataFrame.
- `thousands`: Character to use to detect the thousands separator.

> Full `read_excel` documentation can be found here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html.

In this case we'll try to read our `products.xlsx` Excel file.

This file contains records of products with its price, brand, description and merchant information on different sheets.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Reading our first Excel file

Everytime we call `read_excel` method, we'll need to pass an explicit `filepath` parameter indicating the path where our Excel file is.

Any valid string path is acceptable. The string could be a URL. Valid URL schemes include HTTP, FTP, S3, and file. For file URLs, a host is expected. A local file could be: `file://localhost/path/to/table.xlsx`.

In [ ]:
df = pd.read_excel('data/products.xlsx')

In [ ]:
df.head()

In this case we let pandas infer everything related to our data, but in most of the cases we'll need to explicitly tell pandas how we want our data to be loaded. To do that we use parameters.

Let's see how theses parameters work.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### First row behaviour with `header` parameter

The Excel file we're reading has the following columns:

- `product_id`
- `price`
- `merchant_id`
- `brand`
- `name`

The first row (0-index) of the data has that column names, so we keep the implicit `header=0` parameter to let Pandas assign this first row as headers. We can overwrite this behavior defining explicitly the `header` parameter.

In [ ]:
pd.read_excel('data/products.xlsx').head()

In [ ]:
pd.read_excel('data/products.xlsx',
              header=None).head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Adding index to our data using `index_col` parameter

By default, pandas will automatically assign a numeric autoincremental index or row label starting with zero.

You may want to leave the default index as such if your data doesn’t have a column with unique values that can serve as a better index.

In case there is a column that you feel would serve as a better index, you can override the default behavior by setting `index_col` property to a column. It takes a numeric value or a string for setting a single column as index or a list of numeric values for creating a multi-index.

In our data, we are choosing the first column, `product_id`, as index (index=0) by passing zero to the `index_col` argument.

In [ ]:
df = pd.read_excel('data/products.xlsx',
                   index_col=[0])

In [ ]:
df.head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Selecting specific sheets

Excel files quite often have multiple sheets and the ability to read a specific sheet or all of them is very important. To make this easy, the pandas `read_excel` method takes an argument called `sheet_name` that tells pandas which sheet to read in the data from.

For this, you can either use the sheet name or the sheet number. Sheet numbers start with zero. The first sheet will be the one loaded by default. You can change sheet by specifying `sheet_name` parameter.

In [ ]:
products = pd.read_excel('data/products.xlsx',
                         sheet_name='Products',
                         index_col='product_id')

In [ ]:
products.head()

In [ ]:
merchants = pd.read_excel('data/products.xlsx',
                          sheet_name='Merchants',
                          index_col='merchant_id')

In [ ]:
merchants.head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### The `ExcelFile` class

Another approach on reading Excel data is using the `ExcelFile` class for parsing tabular Excel sheets into `DataFrame` objects.

This `ExcelFile` will let us work with sheets easily, and will be faster than the previous `read_excel` method.

In [ ]:
excel_file = pd.ExcelFile('data/products.xlsx')

In [ ]:
excel_file

We can now explore the sheets on that Excel file with `sheet_names`:

In [ ]:
excel_file.sheet_names

And parse specified sheet(s) into a Pandas' `DataFrame` using ExcelFile's `parse()` method.

Everytime we call `parse()` method, we'll need to pass an explicit `sheet_name` parameter indicating which sheet from the Excel file we want to be parsed. First sheet will be parsed by default.

In [ ]:
products = excel_file.parse('Products')

In [ ]:
products.head()

This `parse()` method has all the parameters we saw before on `read_excel()` method, let's try some of them:

In [ ]:
products = excel_file.parse(sheet_name='Products',
                            header=0,
                            index_col='product_id')

In [ ]:
products.head()

In [ ]:
products.dtypes

In [ ]:
merchants = excel_file.parse('Merchants',
                             index_col='merchant_id')

In [ ]:
merchants.head()

In [ ]:
merchants.dtypes

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Save to Excel file

Finally we can save our `DataFrame` as a Excel file.

In [ ]:
products.head()

A fast, simple way to write a single `DataFrame` to an Excel file is to use the `to_excel()` method of the `DataFrame` directly.

Note that it's required to pass a output file path.

> The `OpenPyXL - openpyxl` library should be installed in order to save Excel files. `pip install openpyxl`

In [ ]:
products.to_excel('data/out.xlsx')

In [ ]:
pd.read_excel('data/out.xlsx').head()

We can specify the sheet name with `sheet_name` parameter:

In [ ]:
products.to_excel('data/out.xlsx',
                  sheet_name='Products')

Further calls of `to_excel` with different sheet names will only overwrite the first sheet instead of adding additional sheets.

Also, be aware that by removing the index, we'll lose that column.

In [ ]:
products.to_excel('data/out.xlsx',
                  index=None)

In [ ]:
pd.read_excel('data/out.xlsx').head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Positioning Data with `startrow` and `startcol`

Suppose we wanted to insert the our data into the spreadsheet file in a position somewhere other than the top-left corner.

We can shift where the `to_excel` method writes the data by using `startrow` to set the cell after which the first row will be printed, and `startcol` to set which cell after which the first column will be printed.

<img src="https://user-images.githubusercontent.com/7065401/68594890-41378b80-0477-11ea-9ae4-ff87e5e1128d.png" />

In [ ]:
products.to_excel('data/out.xlsx',
                  sheet_name='Products',
                  startrow=1,
                  startcol=2)

<img src="https://user-images.githubusercontent.com/7065401/68594828-249b5380-0477-11ea-87d7-af694c09f2d2.png" />

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Saving multiple sheets

If we wanted to write a single `DataFrame` to a single sheet with default formatting then we are done. However, if we want to write multiple sheets and/or multiple `DataFrame`s, then we will need to create an `ExcelWriter` object.

The `ExcelWriter` object is included in the Pandas module and is used to open Excel files and handle write operations. This object behaves almost exactly like the vanilla Python `open` object that we used on previous courses and can be used within a `with` block.

> When the `ExcelWriter` object is executed, any existing file with the same name as the output file will be overwritten.

In [ ]:
writer = pd.ExcelWriter('data/out.xlsx')

In [ ]:
writer

Instead of including the file pathname in the `to_excel` call, we will use the `ExcelWriter` object `writer` instead.

In [ ]:
with writer:
    products.to_excel(writer, sheet_name='Products')

In [ ]:
pd.read_excel('data/out.xlsx', sheet_name='Products').head()

We can now add another `Merchants` sheet simply using the `writer` object:

In [ ]:
with writer:
    merchants.to_excel(writer, sheet_name='Merchants')

In [ ]:
pd.read_excel('data/out.xlsx', sheet_name='Products').head()

In [ ]:
pd.read_excel('data/out.xlsx', sheet_name='Merchants').head()

Or we can save multiple sheets at the same time:

In [ ]:
with pd.ExcelWriter('data/out.xlsx') as writer:
    products.to_excel(writer, sheet_name='Products')
    merchants.to_excel(writer, sheet_name='Merchants')

In that case the resulting `out.xlxs` file will have two sheets `Products` and `Merchants`.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Reading HTML tables

In this lecture we'll learn how to read and parse HTML tables from websites into a list of `DataFrame` objects to work with.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Parsing raw HTML strings

Another useful pandas method is `read_html()`. This method will read HTML tables from a given URL, a file-like object, or a raw string containing HTML, and return a list of `DataFrame` objects.

Let's try to read the following `html_string` into a `DataFrame`.

_(Open data/sample.html for the working example)_

In [ ]:
html_string = """
<table>
    <thead>
      <tr>
        <th>Order date</th>
        <th>Region</th> 
        <th>Item</th>
        <th>Units</th>
        <th>Unit cost</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>1/6/2018</td>
        <td>East</td> 
        <td>Pencil</td>
        <td>95</td>
        <td>1.99</td>
      </tr>
      <tr>
        <td>1/23/2018</td>
        <td>Central</td> 
        <td>Binder</td>
        <td>50</td>
        <td>19.99</td>
      </tr>
      <tr>
        <td>2/9/2018</td>
        <td>Central</td> 
        <td>Pencil</td>
        <td>36</td>
        <td>4.99</td>
      </tr>
      <tr>
        <td>3/15/2018</td>
        <td>West</td> 
        <td>Pen</td>
        <td>27</td>
        <td>19.99</td>
      </tr>
    </tbody>
</table>
"""

In [ ]:
from IPython.core.display import display, HTML
display(HTML(html_string))

In [ ]:
dfs = pd.read_html(html_string)

The `read_html` just returned one `DataFrame` object:

In [ ]:
len(dfs)

In [ ]:
df = dfs[0]

df

Previous `DataFrame` looks quite similar to the raw HTML table, but now we have a `DataFrame` object, so we can apply any pandas operation we want to it.

In [ ]:
df.shape

In [ ]:
df.loc[df['Region'] == 'Central']

In [ ]:
df.loc[df['Units'] > 35]

#### Defining header

Pandas will automatically find the header to use thanks to the <thead> tag.
    
But in many cases we'll find wrong or incomplete tables that make the `read_html` method parse the tables in a wrong way without the proper headers.

To fix them we can use the `header` parameter.

In [ ]:
html_string = """
<table>
  <tr>
    <td>Order date</td>
    <td>Region</td> 
    <td>Item</td>
    <td>Units</td>
    <td>Unit cost</td>
  </tr>
  <tr>
    <td>1/6/2018</td>
    <td>East</td> 
    <td>Pencil</td>
    <td>95</td>
    <td>1.99</td>
  </tr>
  <tr>
    <td>1/23/2018</td>
    <td>Central</td> 
    <td>Binder</td>
    <td>50</td>
    <td>19.99</td>
  </tr>
  <tr>
    <td>2/9/2018</td>
    <td>Central</td> 
    <td>Pencil</td>
    <td>36</td>
    <td>4.99</td>
  </tr>
  <tr>
    <td>3/15/2018</td>
    <td>West</td> 
    <td>Pen</td>
    <td>27</td>
    <td>19.99</td>
  </tr>
</table>
"""

In [ ]:
pd.read_html(html_string)[0]

In this case, we'll need to pass the row number to use as header using the `header` parameter.

In [ ]:
pd.read_html(html_string, header=0)[0]

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Parsing HTML tables from the web

Now that we know how `read_html` works, go one step beyond and try to parse HTML tables directly from an URL.

To do that we'll call the `read_html` method with an URL as paramter.

#### Simple example

In [ ]:
html_url = "https://www.basketball-reference.com/leagues/NBA_2019_per_game.html"

In [ ]:
nba_tables = pd.read_html(html_url)

In [ ]:
len(nba_tables)

We'll work with the only one table found:

In [ ]:
nba = nba_tables[0]

In [ ]:
nba.head()

In [ ]:
nba.head(25)

#### Complex example

We can also use the `requests` module to get HTML code from an URL to parse it into `DataFrame` objects.

If we look at the given URL we can see multiple tables about The Simpsons TV show.

We want to keep the table with information about each season.

In [ ]:
import requests

html_url = "https://en.wikipedia.org/wiki/The_Simpsons"

In [ ]:
r = requests.get(html_url)

wiki_tables = pd.read_html(r.text, header=0)

In [ ]:
len(wiki_tables)

In [ ]:
simpsons = wiki_tables[1]

In [ ]:
simpsons.head()

Quick clean on the table: remove extra header rows and set `Season` as index.

In [ ]:
simpsons.drop([0, 1], inplace=True)

In [ ]:
simpsons.set_index('Season', inplace=True)

Which season has the lowest number of episodes?

In [ ]:
simpsons['No. ofepisodes'].unique()

In [ ]:
simpsons = simpsons.loc[simpsons['No. ofepisodes'] != 'TBA']

In [ ]:
min_season = simpsons['No. ofepisodes'].min()

min_season

In [ ]:
simpsons.loc[simpsons['No. ofepisodes'] == min_season]

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Save to CSV file

Finally save the `DataFrame` to a CSV file as we saw on previous lectures.

In [ ]:
simpsons.head()

In [ ]:
simpsons.to_csv('out-html.csv')

In [ ]:
pd.read_csv('out-html.csv', index_col='Season').head()

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)